In [1]:
!pip install gurunudi

## Gurunudi (Not working due to internal maintenance)

In [2]:
from gurunudi import AI,lang
ai=AI()

### Auto-Correct

In [3]:
corrected_text = ai.autocorrect("ordr stutas 23658")

AttributeError: 'AI' object has no attribute 'autocorrect'

In [ ]:
corrected_text

In [5]:
ai.autocorrect("whic is the fast car")

AttributeError: 'AI' object has no attribute 'autocorrect'

### Auto-Complete

In [6]:
completed_text = ai.autocomplete("which is the fastest ")
completed_text

AttributeError: 'AI' object has no attribute 'autocomplete'

In [7]:
ai.chat("como estas")

APIError: status_code_403

## Spell Check: Peter Norvig

In [9]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [10]:
correction('speling')

'spelling'

In [11]:
correction('korrectud')

'corrected'

## TextBlob

Spelling correction is based on Peter Norvig’s “How to Write a Spelling Corrector”[1] as implemented in the pattern library. It is about 70% accurate

In [12]:
from textblob import TextBlob
b = TextBlob("I havv goood speling!")
print(b.correct())

I have good spelling!


In [13]:
str(TextBlob('sqelinr').correct())

'sqelinr'

## autocorrect

In [14]:
!pip install autocorrect

     |████████████████████████████████| 622 kB 2.2 MB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=6be8806f10b9633c85e8c1bfb7db28719e8df425a6e2843d0d84347d0499a203
  Stored in directory: /home/avinash/.cache/pip/wheels/ab/0f/23/3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect


In [15]:
from autocorrect import Speller
spell = Speller()
print(spell('ordre stutas'))

ordre status


In [16]:
from autocorrect import Speller
spell = Speller()
print(spell('caaaar'))
print(spell(u'mussage'))
print(spell(u'survice'))
print(spell(u'hte'))

aaaaaa
message
service
the


## pyspellchecker

It uses a Levenshtein Distance algorithm to find permutations within an edit distance of 2 from the original word. It then compares all permutations (insertions, deletions, replacements, and transpositions) to known words in a word frequency list. Those words that are found more often in the frequency list are more likely the correct results.

https://buildmedia.readthedocs.org/media/pdf/pyspellchecker/latest/pyspellchecker.pdf 

In [17]:
!pip install pyspellchecker

     |████████████████████████████████| 2.5 MB 2.9 MB/s eta 0:00:01


In [18]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['something', 'is', 'hapenning', 'here', 'korrectid'])

for word in misspelled:
    print("Original word:",word)
    # Get the one `most likely` answer
    print("Correct Word:",spell.correction(word))

    # Get a list of `likely` options
    print("Most Similar Words:",spell.candidates(word))

Original word: korrectid
Correct Word: corrected
Most Similar Words: {'corrected'}
Original word: hapenning
Correct Word: happening
Most Similar Words: {'penning', 'happening', 'henning'}


## SymSpell

In [19]:
!pip install symspellpy

     |████████████████████████████████| 2.6 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.1 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp39-cp39-linux_x86_64.whl size=32299 sha256=93f813eac488ffa48e18edf6c799bd33810840bc29ea6838e9a0dd5aa79bbec2
  Stored in directory: /home/avinash/.cache/pip/wheels/72/16/b1/ea6472dedd6ac13ad789dd294871a76fd6906fca3c010347d1
Successfully built editdistpy


In [20]:
from collections import Counter
from symspellpy import SymSpell
from sklearn.datasets import fetch_20newsgroups
import re
corpus = []
for line in fetch_20newsgroups().data:
    line = line.replace('\n', ' ').replace('\t', ' ').lower()
    line = re.sub('[^a-z ]', ' ', line)
    tokens = line.split(' ')
    tokens = [token for token in tokens if len(token) > 0]
    corpus.extend(tokens)
corpus = Counter(corpus)
corpus_dir = '../../data/'
corpus_file_name = 'spell_check_dictionary.txt'
symspell = SymSpell()
symspell.build_vocab(
    dictionary=corpus, 
    file_dir=corpus_dir, file_name=corpus_file_name)
symspell.load_vocab(corpus_file_path=corpus_dir+corpus_file_name)

AttributeError: 'SymSpell' object has no attribute 'build_vocab'

## language_check

In [21]:
!pip install language-check

  ERROR: Command errored out with exit status 1:
   command: /home/avinash/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ofl_r55r/language-check_74ec1c3c08a546fc8af516c3a9523761/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ofl_r55r/language-check_74ec1c3c08a546fc8af516c3a9523761/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-6sv3tr9y
       cwd: /tmp/pip-install-ofl_r55r/language-check_74ec1c3c08a546fc8af516c3a9523761/
  Complete output (35 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-install-ofl_r55r/language-check_74ec1c3c08a546fc8af516c3a9523761/setup.py", line 595, in <module>
      sys.exit(main())
    File "/t

In [16]:
import language_check
tool = language_check.LanguageTool('en-US')
text = u'A sentence with a error in the Hitchhiker’s Guide tot he Galaxy'
matches = tool.check(text)
len(matches)

2

In [60]:
matches

[Match({'fromy': 0, 'fromx': 16, 'toy': 0, 'tox': 17, 'ruleId': 'EN_A_VS_AN', 'msg': "Use 'an' instead of 'a' if the following word starts with a vowel sound, e.g. 'an article', 'an hour'", 'replacements': ['an'], 'context': 'A sentence with a error in the Hitchhiker’s Guide tot he ...', 'contextoffset': 16, 'offset': 16, 'errorlength': 1, 'category': 'Miscellaneous', 'locqualityissuetype': 'misspelling'}),
 Match({'fromy': 0, 'fromx': 50, 'toy': 0, 'tox': 56, 'ruleId': 'TOT_HE', 'subId': '1', 'msg': "Did you mean 'to the'?", 'replacements': ['to the'], 'context': '... with a error in the Hitchhiker’s Guide tot he Galaxy', 'contextoffset': 43, 'offset': 50, 'errorlength': 6, 'category': 'Possible Typo', 'locqualityissuetype': 'misspelling'})]

In [17]:
for error in matches:
    print(error.replacements, error.msg)

['an'] Use 'an' instead of 'a' if the following word starts with a vowel sound, e.g. 'an article', 'an hour'
['to the'] Did you mean 'to the'?


## Gingerit

In [22]:
!pip install gingerit

In [24]:
from gingerit.gingerit import GingerIt

text = 'The smelt of fliwers bring back memories.'

parser = GingerIt()
parser.parse(text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Train your model for spell checker using TextBlob

Reference: https://www.peterbe.com/plog/train-your-own-spell-corrector-with-textblob 

In [28]:
names=['Liam', 'Noah', 'William', 'James', 'Oliver', 'Benjamin', 'Elijah', 'Lucas', 'Mason', 'Logan', 'Alexander', 'Ethan', 'Jacob', 'Michael', 'Daniel', 'Henry', 'Jackson', 'Sebastian', 'Aiden', 'Matthew', 'Samuel', 'David', 'Joseph', 'Carter', 'Owen', 'Wyatt', 'John', 'Jack', 'Luke', 'Jayden', 'Dylan', 'Grayson', 'Levi', 'Isaac', 'Gabriel', 'Julian', 'Mateo', 'Anthony', 'Jaxon', 'Lincoln', 'Joshua', 'Christopher', 'Andrew', 'Theodore', 'Caleb', 'Ryan', 'Asher', 'Nathan', 'Thomas', 'Leo', 'Isaiah', 'Charles', 'Josiah', 'Hudson', 'Christian', 'Hunter', 'Connor', 'Eli', 'Ezra', 'Aaron', 'Landon', 'Adrian', 'Jonathan', 'Nolan', 'Jeremiah', 'Easton', 'Elias', 'Colton', 'Cameron', 'Carson', 'Robert', 'Angel', 'Maverick', 'Nicholas', 'Dominic', 'Jaxson', 'Greyson', 'Adam', 'Ian', 'Austin', 'Santiago', 'Jordan', 'Cooper', 'Brayden', 'Roman', 'Evan', 'Ezekiel', 'Xavier', 'Jose', 'Jace', 'Jameson', 'Leonardo', 'Bryson', 'Axel', 'Everett', 'Parker', 'Kayden', 'Miles', 'Sawyer', 'Jason', 'Declan', 'Weston', 'Micah', 'Ayden', 'Wesley', 'Luca', 'Vincent', 'Damian', 'Zachary', 'Silas', 'Gavin', 'Chase', 'Kai', 'Emmett', 'Harrison', 'Nathaniel', 'Kingston', 'Cole', 'Tyler', 'Bennett', 'Bentley', 'Ryker', 'Tristan', 'Brandon', 'Kevin', 'Luis', 'George', 'Ashton', 'Rowan', 'Braxton', 'Ryder', 'Gael', 'Ivan', 'Diego', 'Maxwell', 'Max', 'Carlos', 'Kaiden', 'Juan', 'Maddox', 'Justin', 'Waylon', 'Calvin', 'Giovanni', 'Jonah', 'Abel', 'Jayce', 'Jesus']

In [29]:
import random
from string import ascii_lowercase

def get_random_name():
    """ Select random name form names list and make a typo error"""
    random_name = correct_name = random.choice(names).lower()

    random_letters = random.sample(list(ascii_lowercase), len(ascii_lowercase))
    for letter in random_letters:
        if letter in random_name:
            if random.random() > 0.5:
                # removal
                random_name = random_name.replace(letter, "", 1)
            elif random.random() > 0.5:
                # injection
                letters = list(random_name)
                letters.insert(random.randint(0, len(letters)), letter)
                random_name = "".join(letters)
            else:
                # swap
                p = random.randint(0, len(random_name) - 2)
                letters = list(random_name)
                a, b = letters[p : p + 2]
                letters[p] = b
                letters[p + 1] = a
                random_name = "".join(letters)
            break
    return correct_name, random_name

In [30]:
from textblob import TextBlob

correct, typo = get_random_name()
correct, typo

('axel', 'ael')

In [31]:
b = TextBlob(typo)
result = str(b.correct())
right = correct == result

In [33]:
result

'all'

In [32]:
right,typo,correct,result

(False, 'ael', 'axel', 'all')

In [46]:
from textblob.en import Spelling

# file name with path
filename = "spelling-model.txt"
spelling = Spelling(path=filename)


# training the model
spelling.train(" ".join(names), filename)

In [47]:
# correct, typo = get_random_name()
correct, typo='weston', 'wston'
b = spelling.suggest(typo)
result = b[0][0]
right = correct == result
right,typo,correct,result

(True, 'wston', 'weston', 'weston')

In [48]:
spelling.suggest(typo)[0][0]

'weston'

I have changed weight for `adrian`=8 `damian`=3 in spelling-model.txt file and than I am check for `darian` word and got probability score. This weighted scoring spell suggestion.

In [50]:
spelling = Spelling(path=filename)
b = spelling.suggest('darian')
b

[('adrian', 0.7272727272727273), ('damian', 0.2727272727272727)]